In [4]:
file_path = r"C:\Users\olona\Downloads\gameofthrones.txt"
with open(file_path, "r", encoding="utf-8") as file:
    text = file.read()


In [5]:
start_index = 0  # Starting index of the desired text segment
end_index = 50000  # Ending index of the desired text segment
selected_text = text[start_index:end_index]

In [9]:
import re
import spacy
import numpy as np
from tensorflow import keras


nlp = spacy.load("en_core_web_sm")

# Tokenize the selected text using spaCy
doc = nlp(selected_text)
tokens = [token.text for token in doc]


In [16]:
clean_tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens if token.strip() != '' and not token.isspace()]
clean_tokens = list(filter(lambda x: x.strip(), clean_tokens))

print(clean_tokens)

['We', 'should', 'start', 'back', 'Gared', 'urged', 'as', 'the', 'woods', 'began', 'to', 'grow', 'dark', 'around', 'them', 'The', 'wildlings', 'are', 'dead', 'Do', 'the', 'dead', 'frighten', 'you', 'Ser', 'Waymar', 'Royce', 'asked', 'with', 'just', 'the', 'hint', 'of', 'a', 'smile', 'Gared', 'did', 'not', 'rise', 'to', 'the', 'bait', 'He', 'was', 'an', 'old', 'man', 'past', 'fifty', 'and', 'he', 'had', 'seen', 'the', 'lordlings', 'come', 'and', 'go', 'Dead', 'is', 'dead', 'he', 'said', 'We', 'have', 'no', 'business', 'with', 'the', 'dead', 'Are', 'they', 'dead', 'Royce', 'asked', 'softly', 'What', 'proof', 'have', 'we', 'Will', 'saw', 'them', 'Gared', 'said', 'If', 'he', 'says', 'they', 'are', 'dead', 'that', 's', 'proof', 'enough', 'for', 'me', 'Will', 'had', 'known', 'they', 'would', 'drag', 'him', 'into', 'the', 'quarrel', 'sooner', 'or', 'later', 'He', 'wished', 'it', 'had', 'been', 'later', 'rather', 'than', 'sooner', 'My', 'mother', 'told', 'me', 'that', 'dead', 'men', 'sing', 'n

In [17]:
len(clean_tokens)

9385

In [20]:
seq_len = 25+1
text_sequences = []

for i in range(seq_len, len(clean_tokens)):
    
    # Grab train_len# amount of characters
    seq = clean_tokens[i-seq_len:i]
    
    # Add to list of sequences
    text_sequences.append(seq)

In [22]:
type(text_sequences)

list

In [25]:
' '.join(text_sequences[0])

'We should start back Gared urged as the woods began to grow dark around them The wildlings are dead Do the dead frighten you Ser Waymar'

In [26]:
' '.join(text_sequences[1])

'should start back Gared urged as the woods began to grow dark around them The wildlings are dead Do the dead frighten you Ser Waymar Royce'

In [27]:
from keras.preprocessing.text import Tokenizer

In [28]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences)

In [29]:
sequences[0]

[63,
 271,
 1849,
 80,
 42,
 845,
 15,
 1,
 321,
 413,
 7,
 412,
 132,
 102,
 28,
 1,
 563,
 74,
 60,
 84,
 1,
 60,
 1843,
 14,
 57,
 58]

In [30]:
for i in sequences[0]:
    print(f'{i} : {tokenizer.index_word[i]}')

63 : we
271 : should
1849 : start
80 : back
42 : gared
845 : urged
15 : as
1 : the
321 : woods
413 : began
7 : to
412 : grow
132 : dark
102 : around
28 : them
1 : the
563 : wildlings
74 : are
60 : dead
84 : do
1 : the
60 : dead
1843 : frighten
14 : you
57 : ser
58 : waymar


In [31]:
tokenizer.word_counts

OrderedDict([('we', 573),
             ('should', 132),
             ('start', 3),
             ('back', 472),
             ('gared', 811),
             ('urged', 32),
             ('as', 2139),
             ('the', 15983),
             ('woods', 113),
             ('began', 88),
             ('to', 4327),
             ('grow', 90),
             ('dark', 299),
             ('around', 378),
             ('them', 1185),
             ('wildlings', 69),
             ('are', 538),
             ('dead', 613),
             ('do', 462),
             ('frighten', 23),
             ('you', 2182),
             ('ser', 649),
             ('waymar', 624),
             ('royce', 650),
             ('asked', 390),
             ('with', 1950),
             ('just', 182),
             ('hint', 26),
             ('of', 4914),
             ('a', 5846),
             ('smile', 208),
             ('did', 832),
             ('not', 1664),
             ('rise', 78),
             ('bait', 26),
             ('h

In [32]:
vocabulary_size = len(tokenizer.word_counts)
vocabulary_size

1849

In [33]:
import numpy as np

In [34]:
sequences = np.array(sequences)

In [35]:
sequences

array([[  63,  271, 1849, ...,   14,   57,   58],
       [ 271, 1849,   80, ...,   57,   58,   54],
       [1849,   80,   42, ...,   58,   54,   95],
       ...,
       [1842,    5,  618, ...,    1, 1848,  322],
       [   5,  618,    4, ..., 1848,  322,   12],
       [ 618,    4,  411, ...,  322,   12,  414]])

In [36]:
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

In [37]:
def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, 25, input_length=seq_len))
    model.add(LSTM(150, return_sequences=True))
    model.add(LSTM(150))
    model.add(Dense(150, activation='relu'))

    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    model.summary()
    
    return model

In [38]:
from keras.utils import to_categorical

In [39]:
sequences

array([[  63,  271, 1849, ...,   14,   57,   58],
       [ 271, 1849,   80, ...,   57,   58,   54],
       [1849,   80,   42, ...,   58,   54,   95],
       ...,
       [1842,    5,  618, ...,    1, 1848,  322],
       [   5,  618,    4, ..., 1848,  322,   12],
       [ 618,    4,  411, ...,  322,   12,  414]])

In [40]:
# First 24 words
sequences[:,:-1]

array([[  63,  271, 1849, ..., 1843,   14,   57],
       [ 271, 1849,   80, ...,   14,   57,   58],
       [1849,   80,   42, ...,   57,   58,   54],
       ...,
       [1842,    5,  618, ...,  162,    1, 1848],
       [   5,  618,    4, ...,    1, 1848,  322],
       [ 618,    4,  411, ..., 1848,  322,   12]])

In [42]:
sequences[:,-1]

array([ 58,  54,  95, ..., 322,  12, 414])

In [43]:
X = sequences[:,:-1]

In [44]:
y = sequences[:,-1]

In [45]:
y = to_categorical(y, num_classes=vocabulary_size+1)

In [46]:
seq_len = X.shape[1]

In [47]:
X.shape[1]

25

In [48]:
# define model
model = create_model(vocabulary_size+1, seq_len)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 25)            46250     
                                                                 
 lstm (LSTM)                 (None, 25, 150)           105600    
                                                                 
 lstm_1 (LSTM)               (None, 150)               180600    
                                                                 
 dense (Dense)               (None, 150)               22650     
                                                                 
 dense_1 (Dense)             (None, 1850)              279350    
                                                                 
Total params: 634,450
Trainable params: 634,450
Non-trainable params: 0
_________________________________________________________________


In [49]:
from pickle import dump,load

In [50]:
model.fit(X, y, batch_size=128, epochs=300,verbose=1)

Epoch 1/300
74/74 [==============================] - 30s 151ms/step - loss: 6.6050 - accuracy: 0.0514
Epoch 2/300
74/74 [==============================] - 11s 149ms/step - loss: 6.1940 - accuracy: 0.0657
Epoch 3/300
74/74 [==============================] - 12s 160ms/step - loss: 6.1682 - accuracy: 0.0657
Epoch 4/300
74/74 [==============================] - 11s 150ms/step - loss: 6.1139 - accuracy: 0.0657
Epoch 5/300
74/74 [==============================] - 11s 151ms/step - loss: 6.0052 - accuracy: 0.0657
Epoch 6/300
74/74 [==============================] - 11s 155ms/step - loss: 5.9040 - accuracy: 0.0676
Epoch 7/300
74/74 [==============================] - 11s 154ms/step - loss: 5.7811 - accuracy: 0.0689
Epoch 8/300
74/74 [==============================] - 12s 156ms/step - loss: 5.6974 - accuracy: 0.0744
Epoch 9/300
74/74 [==============================] - 12s 159ms/step - loss: 5.6200 - accuracy: 0.0767
Epoch 10/300
74/74 [==============================] - 12s 159ms/step - loss: 5.556

74/74 [==============================] - 12s 159ms/step - loss: 3.2554 - accuracy: 0.2245
Epoch 81/300
74/74 [==============================] - 12s 157ms/step - loss: 3.2341 - accuracy: 0.2345
Epoch 82/300
74/74 [==============================] - 11s 146ms/step - loss: 3.2252 - accuracy: 0.2309
Epoch 83/300
74/74 [==============================] - 12s 157ms/step - loss: 3.1972 - accuracy: 0.2374
Epoch 84/300
74/74 [==============================] - 12s 157ms/step - loss: 3.1694 - accuracy: 0.2483
Epoch 85/300
74/74 [==============================] - 12s 156ms/step - loss: 3.1462 - accuracy: 0.2515
Epoch 86/300
74/74 [==============================] - 11s 148ms/step - loss: 3.1327 - accuracy: 0.2445
Epoch 87/300
74/74 [==============================] - 11s 150ms/step - loss: 3.1073 - accuracy: 0.2540
Epoch 88/300
74/74 [==============================] - 11s 147ms/step - loss: 3.0845 - accuracy: 0.2601
Epoch 89/300
74/74 [==============================] - 11s 144ms/step - loss: 3.0618 - 

74/74 [==============================] - 11s 149ms/step - loss: 1.8843 - accuracy: 0.5130
Epoch 160/300
74/74 [==============================] - 11s 151ms/step - loss: 1.8554 - accuracy: 0.5196
Epoch 161/300
74/74 [==============================] - 11s 150ms/step - loss: 1.8439 - accuracy: 0.5200
Epoch 162/300
74/74 [==============================] - 11s 150ms/step - loss: 1.8293 - accuracy: 0.5241
Epoch 163/300
74/74 [==============================] - 11s 147ms/step - loss: 1.8206 - accuracy: 0.5286
Epoch 164/300
74/74 [==============================] - 11s 148ms/step - loss: 1.8049 - accuracy: 0.5357
Epoch 165/300
74/74 [==============================] - 11s 153ms/step - loss: 1.8043 - accuracy: 0.5273
Epoch 166/300
74/74 [==============================] - 11s 148ms/step - loss: 1.7711 - accuracy: 0.5429
Epoch 167/300
74/74 [==============================] - 11s 145ms/step - loss: 1.7603 - accuracy: 0.5384
Epoch 168/300
74/74 [==============================] - 11s 150ms/step - loss: 

74/74 [==============================] - 11s 145ms/step - loss: 0.8730 - accuracy: 0.7893
Epoch 238/300
74/74 [==============================] - 11s 155ms/step - loss: 0.8584 - accuracy: 0.7920
Epoch 239/300
74/74 [==============================] - 12s 166ms/step - loss: 0.8289 - accuracy: 0.7957
Epoch 240/300
74/74 [==============================] - 11s 143ms/step - loss: 0.8312 - accuracy: 0.7970
Epoch 241/300
74/74 [==============================] - 11s 151ms/step - loss: 0.8121 - accuracy: 0.8048
Epoch 242/300
74/74 [==============================] - 13s 173ms/step - loss: 0.8046 - accuracy: 0.8076
Epoch 243/300
74/74 [==============================] - 13s 172ms/step - loss: 0.7997 - accuracy: 0.8122
Epoch 244/300
74/74 [==============================] - 13s 173ms/step - loss: 0.7873 - accuracy: 0.8119
Epoch 245/300
74/74 [==============================] - 13s 169ms/step - loss: 0.7837 - accuracy: 0.8094
Epoch 246/300
74/74 [==============================] - 12s 165ms/step - loss: 

In [51]:
# save the model to file
model.save('firstmodel.h5')
# save the tokenizer
dump(tokenizer, open('firstmodel', 'wb'))

In [54]:
from random import randint
from pickle import load
from keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [63]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    '''
    INPUTS:
    model : model that was trained on text data
    tokenizer : tokenizer that was fit on text data
    seq_len : length of training sequence
    seed_text : raw string text to serve as the seed
    num_gen_words : number of words to be generated by model
    '''
    
    # Final Output
    output_text = []
    
    # Intial Seed Sequence
    input_text = seed_text
    
    # Create num_gen_words
    for i in range(num_gen_words):
        
        # Take the input text string and encode it to a sequence
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        # Pad sequences to our trained rate (50 words in the video)
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        
        # Predict Class Probabilities for each word
        pred_word = model.predict(pad_encoded, verbose=0)[0]
        
        predicted_classes = pred_word.argmax(axis=-1)
 
        
        # Grab word
        pred_word = tokenizer.index_word[predicted_classes] 
        
        # Update the sequence of input text (shifting one over with the new word)
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
        
    # Make it look like a sentence.
    return ' '.join(output_text)

In [64]:
text_sequences[0]

['We',
 'should',
 'start',
 'back',
 'Gared',
 'urged',
 'as',
 'the',
 'woods',
 'began',
 'to',
 'grow',
 'dark',
 'around',
 'them',
 'The',
 'wildlings',
 'are',
 'dead',
 'Do',
 'the',
 'dead',
 'frighten',
 'you',
 'Ser',
 'Waymar']

In [65]:
import random
random.seed(101)
random_pick = random.randint(0,len(text_sequences))

In [66]:
random_seed_text = text_sequences[random_pick]

In [67]:
random_seed_text

['to',
 'hope',
 'They',
 'emerged',
 'silently',
 'from',
 'the',
 'shadows',
 'twins',
 'to',
 'the',
 'first',
 'Three',
 'of',
 'them',
 'four',
 'five',
 'Ser',
 'Waymar',
 'may',
 'have',
 'felt',
 'the',
 'cold',
 'that',
 'came']

In [68]:
seed_text = ' '.join(random_seed_text)

In [69]:
seed_text

'to hope They emerged silently from the shadows twins to the first Three of them four five Ser Waymar may have felt the cold that came'

In [71]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=25)

'with them but he never saw them never heard them will was nt have see it today it was strange never find them as even'